In [82]:
import jieba
import jieba.posseg as pseg
import jieba.analyse
import string
from string import punctuation
import re
from sklearn.neural_network import MLPClassifier
import os
from gensim.models import word2vec
import gensim #这个包好像下载到python 3.10去了，这里估计用的是python3.10
import logging
import numpy as np
from sklearn import svm, metrics
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split

In [87]:
def remove_some(words):
    words = words.replace("\n", "")
    words = words.split("\t")
    while '' in words:
        words.remove('')
    return words

# 创建停用词list
def stopwordslist(filepath):
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]
    return stopwords

# 对句子进行分词
def seg_sentence(sentence):
    sentence_seged = sentence.split("\t")
    stopwords = stopwordslist('stopword.txt')  # 这里加载停用词的路径
    outstr = ''
    for word in sentence_seged:
        if word not in stopwords:
            if word != '\n':
                outstr += word
                outstr += " "
    return outstr

def get_word_vector(path):
    ip = open(path, 'r', encoding='utf-8')
    content = ip.readlines()
    #print(content)
    vecs = []

    for words in content:
        # vec = np.zeros(2).reshape((1, 2))
        vec = np.zeros(50).reshape((1, 50))
        count = 0
        # word = word.split("\t")
        words = remove_some(words)
        for word in words:
            #print("*****"+str(word))
            try:
                count += 1
                # vec += model[word].reshape((1, 2))
                vec += model.wv[word].reshape((1, 50)) #.reshape((1, 50)) 
                # print(vec)
            except KeyError:
                continue
        vec /= count
        vecs.append(vec)
    return vecs

In [106]:

# generate_data("./ttt1.txt")
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
sentences = word2vec.Text8Corpus(u"./tt_digital.txt")  # 加载语料
if os.path.exists("./model_digital"):
    model = gensim.models.Word2Vec.load('./model_digital')
else:
    model = word2vec.Word2Vec(sentences, min_count=1,vector_size=50)  # 训练skip-gram模型 
    model.save("./model_book")
# sentences = generate_data("./data11")
normal_path = open('./digital_positive_word_list.txt', 'r', encoding='utf-8')
spam_path = open('./digital_negative_word_list.txt', 'r', encoding='utf-8')
normal = get_word_vector('./digital_positive_word_list.txt')
spam = get_word_vector('./digital_negative_word_list.txt')

# normal = extendd(normal)
# spam = extendd(spam)

normal_tag = np.ones((len(normal)))
spam_tag = np.zeros((len(spam)))
# print(normal)
train = normal + spam
train_tag = normal_tag.tolist() + spam_tag.tolist()

X_train, X_test, y_train, y_test = train_test_split(np.array(train, dtype='float64'),
                                                    np.array(train_tag, dtype='float64'), test_size=0.30,
                                                    random_state=0)  # 随机选择30%作为测试集，剩余作为训练集

from sklearn.tree import DecisionTreeClassifier  # 决策树分类器
# clf = DecisionTreeClassifier(max_depth=8)        #神经网络分类器

# clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(100, 100, 100, 100))
# clf = svm.SVC()  # 使用RBF核   高斯核函数
clf = svm.LinearSVC()  # 使用线性核
X_train = np.squeeze(X_train)
X_test = np.squeeze(X_test)
y_train = np.squeeze(y_train)
y_test = np.squeeze(y_test)

   

2023-06-01 10:30:48,787 : INFO : collecting all words and their counts
2023-06-01 10:30:48,793 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2023-06-01 10:30:48,910 : INFO : collected 10493 word types from a corpus of 202932 raw words and 21 sentences
2023-06-01 10:30:48,911 : INFO : Creating a fresh vocabulary
2023-06-01 10:30:48,966 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=1 retains 10493 unique words (100.00% of original 10493, drops 0)', 'datetime': '2023-06-01T10:30:48.966877', 'gensim': '4.3.0', 'python': '3.10.9 (main, Mar  1 2023, 12:33:47) [Clang 14.0.6 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'prepare_vocab'}
2023-06-01 10:30:48,968 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=1 leaves 202932 word corpus (100.00% of original 202932, drops 0)', 'datetime': '2023-06-01T10:30:48.968611', 'gensim': '4.3.0', 'python': '3.10.9 (main, Mar  1 2023, 12:33:47) [Clang 14.0.6 ]', 'platform': 'macOS-10.1

In [84]:
#get_word_vector('./positive_word_list.txt')
?word2vec.Word2Vec

In [107]:
from sklearn.impute import SimpleImputer
imp=SimpleImputer()

X_train = imp.fit_transform(X_train)
X_test = imp.fit_transform(X_test)


clf_res = clf.fit(X_train, y_train)
train_pred = clf_res.predict(X_train)
test_pred = clf_res.predict(X_test)
print(test_pred)
#    accuracy = np.mean(test_pred == y_test)
#   print(accuracy)
#  print(metrics.classification_report(y_test, test_pred, target_names=X_test))



[1. 1. 1. ... 1. 1. 1.]


/Users/hongruoyu/anaconda3/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [109]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

#train
acc_1 = accuracy_score(y_train, train_pred)
p_1=precision_score(y_train, train_pred)
r_1=recall_score(y_train, train_pred)
f1_1=f1_score(y_train, train_pred)

#test
acc_2 = accuracy_score(y_test, test_pred)
p_2=precision_score(y_test, test_pred)
r_2=recall_score(y_test, test_pred)
f1_2=f1_score(y_test, test_pred)

print("digital_svm")
print("train",'acc', acc_1, 'p', p_1, 'r', r_1, 'f1', f1_1)#test

print("==========================================")
print("test",'acc', acc_2, 'p', p_2, 'r', r_2, 'f1', f1_2)#test
#print(acc)
#print(classification_report(y_test, test_pred))

digital_svm
train acc 0.5071428571428571 p 0.5038676020867062 r 0.9992864787727435 f1 0.6699354221478115
test acc 0.505 p 0.501889962200756 r 0.9983291562238931 f1 0.6679709334823924
